<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 2: Sistemas de ecuaciones numéricas lineales

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import matplotlib.pyplot as plt

import numpy as np

from scipy.linalg import orth

In [ ]:
from anmi.genericas import norma, print_verbose, norma_1, norma_inf, norma_2

from numpy.linalg import cond as numero_condicion
from anmi.T2 import descomposicion_LU, descomposicion_LDU, cholesky, gram_schmidt, householder, factorizacion_QR
from anmi.T2 import metodo_iterativo, criterio_m_matriz, criterio_SOR, criterio_simetrica_definida_positiva, criterio_diagonal_dominante, criterio_radio_espectral

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [ ]:
help(numero_condicion)

In [ ]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

In [ ]:
numero_condicion(np.array(M).astype(int))

In [ ]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

In [ ]:
numero_condicion(M_ort)

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [ ]:
help(descomposicion_LU)

In [ ]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [ ]:
descomposicion_LU(M, permutar_max=False)

In [ ]:
descomposicion_LDU(M, permutar_max=False)

In [ ]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

In [ ]:
descomposicion_LDU(M, permutar_max=True)

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [ ]:
help(cholesky)

In [ ]:
M = mat(((2, -1, 0), (-1, 2, -2), (0, 2, 1)))
M

In [ ]:
cholesky(M)

In [ ]:
cholesky(M + M.T)

In [ ]:
M + M.T

In [ ]:
cholesky(M + M.T) * cholesky(M + M.T).T

In [ ]:
# Podemos hacer también Cholesky a una matriz con símbolos!
x = symbols('x')

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

In [ ]:
cholesky(Mx)

### Ortogonalización de Gram-Schmidt




In [ ]:
help(gram_schmidt)

In [ ]:
M

In [ ]:
GS = gram_schmidt(M)
GS['P']

In [ ]:
GS['Pn']

In [ ]:
GS['c']

In [ ]:
GS['P'][:, 1].T * GS['P'][:, 2]

In [ ]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [ ]:
GSx = gram_schmidt(Mx)
GSx['P']

### Transformación de Householder

La transformación de Householder es una transformación para pasar de un vertor $x$ a un vector $y$. Para ello se toma el vector $e$, que sería el eje de transformación, y la matriz aplicación es $H = I - 2ee^t$.

En este caso, $$e = \pm \frac{x - y}{\vert\vert x - y \vert\vert}$$

Para la transformación de Householder $x$ e $y$ **tienen que tener la misma norma**. El vector resultante $e$ tiene norma 1.


In [ ]:
help(householder)

In [ ]:
v0 = mat(((1, 0)))
vf = mat((1, 1))
vf = vf/norma(vf)

In [ ]:
H, e = householder(v0, vf, normalizar=False)
H

In [ ]:
e

In [ ]:
# Ejercicio 10
t = symbols('t')

v0 = mat(((1, 0)))
vf = mat((cos(t), sin(t)))

H, e = householder(v0, vf, normalizar=False)

In [ ]:
H

In [ ]:
e

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

In [ ]:
help(factorizacion_QR)

In [ ]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [ ]:
factorizacion_QR(M, metodo='householder')

In [ ]:
A1 = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [ ]:
A1 = mat(((1, 2, 3), (4, 5, 6), (7, 8, 9)))
A1

In [ ]:
dqr = factorizacion_QR(A1, metodo='householder')

In [ ]:
dqr['Q']

In [ ]:
dqr['R']

In [ ]:
simplify(dqr['Q'].T * dqr['Q'])

In [ ]:
simplify(dqr['Q'] * dqr['R'])

### Métodos iterativos



Los métodos iterativos son métodos para resolver el sistema $Ax=b$. Todos los métodos se basan en el mismo criterio, que es emplear una matriz $H$ tal que, para una iteración:
$$x^{(k+1)} = Hx^{(k)} + b'$$

Si tomamos la matriz $A = D - L - U$ (donde $D$ es la diagonal de $A$, y $L$ y $U$ son matrices triangulares de elementos de $A$), la ecuación anterior queda:
$$Mx^{(k+1)} = (M-A)x^{(k)} + b$$

Y simplemente hay que despejar para $x^{(k+1)}$ (pues $x^{(k)}$ lo conocemos) y repetir el procedimiento hasta convergencia:
$$x^{(k+1)} = M^{-1}(M-A)x^{(k)} + M^{-1}b$$

Según el método, la matriz difiere: para Jacobi es $M = D$, para Gauss-Seidel es $M = D-L$ y para sobre-relajación (ros) es $M = \frac{1}{\omega}D - L$.

In [ ]:
help(metodo_iterativo)

In [ ]:
A = mat([[2, 1, 1], [1, -2, 1], [1, 1, 2]])
b = mat([[1, 1, 1]]).T

In [ ]:
diter = metodo_iterativo(A, b, n_iter=200, verbose=True)
np.array(diter['x'], dtype=float)

In [ ]:
# EJERCICIO 12
a = symbols('a')

A = mat([[4, 1, a], [1, 4, 1], [a, 1, 4]])
metodo_iterativo(A, metodo='gs', n_iter=3)

In [ ]:
# EJERCICIO 14
A = mat([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
norma_1(A)
norma_inf(A)
d = [simplify(i) for i in list((A.T * A).eigenvals().keys())]
# norma_2(A)

In [ ]:
A

In [ ]:
A.T * A

In [ ]:
simplify((A.T * A - a * eye(3)))

# Ejercicios

#### Ejercicio 14
Calcular $\vert\vert A\vert\vert_1, \vert\vert A\vert\vert_2, \vert\vert A\vert\vert_\infty$ de la matriz

In [ ]:
A = Matrix([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
A

In [ ]:
np.sum(abs(A), 0), norma_1(A)

In [ ]:
np.sum(abs(A), 1), norma_inf(A)

In [ ]:
# Para la norma_2 podemos usar norma_2(), pero vamos a hacerlo a mano. Para ello:
# 1) Hallamos A.T * A
AA = simplify(A.T * A)
AA

In [ ]:
# En este punto los resultados difieren, así que usaremos sus resultados intermedios
AA = Matrix([[65, 4, -32], [4, 80, 8], [-32, 8, 17]]) / 81

In [ ]:
# 2) Hallamos los autovalores
factor(det(AA - symbols('lambda') * eye(3)))

In [ ]:
# Los autovalores son 0 y 1. Luego nos quedamos con 1
rho = max(solve(det(AA - symbols('lambda') * eye(3))))
norma_2 = sqrt(rho)
norma_2

#### Ejercicio 15
La matriz de Hilbert $H_n$ tiene como coeficientes $h_{ij} = \frac{1}{i + j -1}$
Determinar el límite para $\vert\vert \; \vert\vert_1$ y $\vert\vert \; \vert\vert_\infty$: $\lim_{n\to \infty}\vert\vert H_n \vert\vert$.

In [ ]:
def Hn(n):
    hn = zeros(n, n)
    for row in range(n):
        for col in range(n):
            hn[row, col] = S(1) / S(row + col + 1) # porque i y j les sumamos 1

    return hn

In [ ]:
# Vamos a crear unas matrices de prueba

In [ ]:
Hn(7)

Vemos que la matriz es simétrica, luego norma 1 y norma inf serán iguales.
La columna / fila con mayor suma es la primera, pues:
$A[1:, 0] = A[:-1, 1]$ y $A[0, 0] > A[-1, 1]$

Y por inducción en el resto de columnas se ve que $\sum A[:, i] > \sum A[:, i+1]$. 

El valor para $n$ de la primera fila es $\sum A[:, 0] = \sum_i^n 1/i$, que es la serie harmónica.

Para $n \to \infty$, sabemos que $\sum_i^n 1/i \to \infty$, luego $\lim_{n\to \infty}\vert\vert H_n \vert\vert = \infty$

#### Ejercicio 16
Se considera la matriz 

$$ A = \begin{pmatrix}a & 1+a\\0 & a\end{pmatrix}$$

con $a > 0 \in \mathbb{R}$.

1) Calcular el número de condición de $A$ en la norma $\vert \vert A \vert \vert_\infty$$

In [ ]:
# El núemro de condición es ||A||·||A^-1||
a = symbols('a')

A = Matrix([[a, 1+a], [0, a]])
A

In [ ]:
Ainv = A.inv()
Ainv

In [ ]:
# Norma_inf A es a + a + 1 = 1 + 2a
ninfA = 1 + 2*a

# Norma_inf Ainv es 1/a + 1/a + 1/a2 = 
ninfAinv = 1/a + 1/a + 1/a**2

ninfA, ninfAinv

In [ ]:
condicion = ninfA * ninfAinv
simplify(condicion)

In [ ]:
simplify(1 - factor(condicion))

2) Estimar el error relativo de la solución del sistema lineal perturbado
$$(A + \delta A)(x + \delta x) = b + \delta b$$

Por el lema de Banach tenemos que 

$$\frac{\vert\vert \delta x\vert\vert}{\vert\vert x\vert\vert} = \frac{cond(A)}{1-cond(A)\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}}\left(
    \frac{\vert\vert \delta b\vert\vert}{\vert\vert b\vert\vert} + 
    \frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}\right)$$


Para este ejercicio es 
$$\frac{\vert\vert \delta x\vert\vert}{\vert\vert x\vert\vert} = \frac{
    (2a+1)^2
}{(a^2 - (2a+1)^2)\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}}\left(
    \frac{\vert\vert \delta b\vert\vert}{\vert\vert b\vert\vert} + 
    \frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}\right)$$

De lo cual se cumple que 
$$\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert} < \frac{1}{cond(A)}$$

#### Ejercicio 17
Calcular la factorización LU de la matriz $A$ con pivote parcial con $0 < \alpha < \frac{\pi}{4}$.

In [ ]:
a = symbols('alpha')
A = Matrix([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, cos(a), -sin(a)], [0, 0, sin(a), cos(a)]])
A

En este caso tenemos que la función LU nos falla porque el máximo de la matriz "no existe". Vamos a programarlo.


Vamos a emplear la factorización de Doolitle del estilo $PA = LU$. En este caso, $P = P^{(1)}$ 

In [ ]:
P = Matrix([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P 

In [ ]:
dict_LU = descomposicion_LU(P * A, permutar_max=False)

In [ ]:
dict_LU['L']

In [ ]:
dict_LU['U']

In [ ]:
simplify(dict_LU['L'] * dict_LU['U'])

Vemos que tras la primera permutación, la función funciona sin problemas.

### Ejercicio 18

Sea $L$ la matriz con coeficientes $l_{ij} = \max\{i-j+1, 0\}$ para todo $i, j = 1, \cdots, n$. Se sabe que $L$ es triangular inferior que aparece en la factorización de Cholesky de una matriz $A$ de $n$ filas y $n$ columnas, de la que únicamente se sabe que su coeficiente $a_{n2} = 14$. Determinar la dimensión de $A$

Es decir, $a_{n2} = 14$, y $a_{ij} = \sum_{k \le \min\{i,j\}}^n l_{ik}l_{jk}$
Luego $a_{n2} = l_{n1}l_{21} + l_{n2}l_{22} = (n) \cdot (2) + (n - 1) \cdot (1) = 2n + n - 1 = 3n - 1$


Luego $3n - 1 = 14$ y $n = 15/3 = 5$.

Por tanto, la matriz $L$ es:

In [ ]:
L = zeros(5, 5)
for i in range(5):
    for j in range(5):
        ij = max(i - j + 1, 0)
        L[i, j] = ij
L

Y $A$ es

In [ ]:
A = L * L.T
A

Y se ve que $a_{52} = a_{n2} = 14$

### Ejercicio 19

Determinar en que rango los valores de $\alpha$ de la matriz $A$ es invertible y admite una factorización de Cholesky.


In [ ]:
a = Symbol('alpha')

A = Matrix([[2, 1, a], [1, 2, 1], [a, 1, 2]])
A

La matriz es invertible en aquellos casos donde su determinante no sea nulo.


In [ ]:
factor(det(A))

Vemos que las soluciones son $\alpha = 2$ y $\alpha = -1$. En esos casos la matriz no es invertible. Además, existe una factorización de Cholesky para los casos en los que $\det(A) > 0$, que se cuple cuando $1 < \alpha < 2$. En esos casos la factorización es

In [ ]:
cholesky(A)

### Ejercicio 20

Determinar la factorización de Cholesky de $A_{n\times n}$ con coeficientes. 
$$a_{11} = 1$$
$$a_{ii} = 1 + i^2$$
$$a_{i, i+1} = a_{i+1, i} = -i$$

Calcular el determinante de $A$

Vamos a sacar algunos ejemplos ilustrativos para hacernos una idea.

In [ ]:
n = 7
A = zeros(n, n)
for i in range(n):
    for j in range(n):
        if i == j:
            A[i,j] = (i+1) ** 2 + 1
        elif i == (1 + j):
            A[i,j] = -i
        elif j == (1 + i):
            A[i,j] = -j
        else:
            pass
A[0, 0] = 1
A

In [ ]:
cholesky(A)

Así a primera vista vemos el patrón evidente. Vamos a demostrarlo.

Sabemos que $l_{11} = 1$ y $l_{21} = -1$

Vamos a probar un par de casos.

Para $i = 2$ tenemos que
$$l_{22} = \sqrt{a_{22} - l_{21}^2} = 
\sqrt{1 + 2^2 - (-1)^2} = 2$$
$$l_{32} = \frac{a_{32} - \sum_k^1l_{3k}l_{2k}}{l_{22}} = 
 \frac{-2 - (-1)(0)}{2}  =  -1$$
Y $l_{31} = 0$

Para $i=3$ tenemos

$$l_{33} = \sqrt{a_{33} - \sum_{k=1}^{2}l_{3k}^2} = 
\sqrt{1 + 3^2 - l_{32}^2} = \sqrt{1 + 3^2 - (-1)^2} = 3$$
$$l_{43} = \frac{-3 - l_{42}l_{32}}{l_{33}} = 
 \frac{-3 - (-1)(0)}{3}  =  -1$$

Y, por inducción:
$$l_{ii} = \sqrt{a_{ii} - l_{i, i-1}^2} = \sqrt{1 + i^2 - 1} = i$$
$$l_{i+1, i} = \frac{a_{i+1,i}}{l_ii} = \frac{-i}{i} = -1$$


Luego la matriz es de la forma:
$\det{A} = \det({L})\det({L^T}) = \det({L})^2 = (n!)^2$

### Ejercicio 21

Construir una trnaformación de Householder que transforme el vector $x = (\cos\alpha, -\sin\alpha, 1, 1)^t$ en el vector $y = (1, 1, \sin\alpha, \cos\alpha)^t.

Encontrar la matriz asociada a esta transformación para $\alpha = \pi/2$

In [ ]:
vx = Matrix([cos(a), -sin(a), 1, 1])
vy = Matrix([1, 1, sin(a), cos(a)])
vx

In [ ]:
H, e = householder(vx, vy, normalizar=True)
simplify(H)

In [ ]:
simplify(e)

In [ ]:
H.subs(a, pi/2)

### Ejercicio 22

Sean $A$ y $b$. Aplicar el método de Gram-Schmidt para ortonormalizar el conjunto de vectores $\{b, Ab, A^2b\}$.

In [ ]:
A = Matrix([[1, 2, 0], [2, 1, 2], [0, 2, 1]])
b = Matrix([1, -1, 1])

In [ ]:
T = zeros(3, 3)
T[:, 0] = b
T[:, 1] = A * b
T[:, 2] = A ** 2 * b
T

In [ ]:
GS = gram_schmidt(T)
GS['P']

In [ ]:
GS['Pn']

### Ejercicio 23

Se considera el sistem lineal $Ax=b$. Hallar el valor de $\beta$ que minimice el radio espectral de $I - \beta A$

In [ ]:
A = Matrix([[1, 1, 2], [0, 2, -1], [0, -1, 2]])
b = Matrix([1, 1, 1])
beta = Symbol('beta')

Primero vamos a hallar el radio espectral de la matriz $I - \beta A$

In [ ]:
iba = eye(3) - beta * A
iba

In [ ]:
l = Symbol('lambda')
eq = factor(det(iba - eye(3) * l))
eq

Vemos que los autovalores de $I - \beta A$ son $\lambda = 1 - \beta$ y 
$\lambda = 1 - 3\beta$

In [ ]:
x = np.linspace(-1, 1, 100)
plt.plot(x, [max(abs(1-i), abs(1-3*i)) for i in x])

Vemos que el menor valor está en $\beta = 0.5$ donde $\rho(I - \beta A) = 0.5$

### Ejercicio 24

Se considera el sistema lineal 
$$x_{i+1} = x_i + x_{i-1} \;\;\; i = 1, 2, 3$$
$$x_0 = 1, x_4 = 5$$

Es decir:
$$\begin{align}
x_2 & =  x_1 + x_0\\
x_3 & =  x_2 + x_1\\
x_4 & = x_3 + x_2\\
\end{align}
$$

Resolviendo parámetros
$$\begin{align}
x_2 & =  x_1 + 1\\
x_3 & =  x_2 + x_1\\
5 & = x_3 + x_2\\
\end{align}
$$

Analizar la convergencia del método de jacobi cuando se aplica a la resolución de este sistema de ecuaciones.

El sistema de ecuaciones se puede reordenar de la siguiente manera:
$$\begin{align}
-x_1 + x_2 + 0 \cdot x_3 & =  1\\
-x_1 - x_2 + x_3 & =  0\\
0\cdot x_1 - x_2 - x_3 & = 5\\
\end{align}
$$

Que, simplificando signos, queda:
$$
\begin{align}
-x_1 + x_2 + 0  & =  1\\
x_1 + x_2 - x_3 & =  0\\
0 + x_2 + x_3 & = -5\\
\end{align}
$$

En sistema matricial queda como
$$
\begin{pmatrix}
-1 & 1 & 0 \\
1 & 1 & -1 \\
0 & 1 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x_1\\x_2\\x_3
\end{pmatrix} = 
\begin{pmatrix}
1 \\ 0 \\ -5
\end{pmatrix}
$$

In [ ]:
A = Matrix([[-1, 1, 0], [1, 1, -1], [0, 1, 1]])
D = Matrix([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
b = Matrix([1, 0, -5])

In [ ]:
help(metodo_iterativo)

In [ ]:
metodo_iterativo(A=A, b=b, metodo='jacobi')

Por el criterio del radio espectral, tenemos que los autovalores son $\pm\sqrt{2}i$, y el radio espectral es $\sqrt{2} > 1$, luego el método de Jacobi no converge.
De hecho vemos que la matriz de diferencias crece con cada iteración.

### Ejercicio 25

Analizar la convergencia de los métodos de Jacobi, Gauss-Seidel y SOR cuando se aplica el sistema lineal

$$
\begin{pmatrix}
4 & 1 & 0 & \cdots & 0 & 0 & 0 \\
1 & 4 & 1 & \cdots & 0 & 0 & 0 \\
0 & 1 & 4 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 4 & 1 & 0 \\
0 & 0 & 0 & \cdots & 1 & 4 & 1 \\
0 & 0 & 0 & \cdots & 0 & 1 & 4 \\
\end{pmatrix}
\begin{pmatrix}
x_1\\x_2\\x_3\\\vdots\\x_{n-2}\\x_{n-1}\\x_n
\end{pmatrix}
=
\begin{pmatrix}
1\\2\\3\\\vdots\\n-2\\n-1\\n
\end{pmatrix}
$$

In [ ]:
n = 8
A = eye(n) * 4
b = ones(n, 1)
for i in range(n-1):
    A[i+1, i], A[i, i+1], b[i+1, 0] = 1, 1, i+2
A

In [ ]:
b

In [ ]:
dict_jac = metodo_iterativo(A=A, b=b, metodo='jacobi')
N(dict_jac['x']), N(Matrix(dict_jac['diff']))

In [ ]:
dict_gs = metodo_iterativo(A=A, b=b, metodo='gs')
N(dict_gs['x']), N(Matrix(dict_gs['diff']))

In [ ]:
dict_sor = metodo_iterativo(A=A, b=b, metodo='sor')
N(dict_sor['x']), N(Matrix(dict_sor['diff']))

Vemos que las matrices son convergentes, y los criterios inmediatos de convergencia se justifican. Vamos a estudiar cada uno de ellos en detalle.

**Criterio de radio espectral**: vemos que hay autovalores mayores que 1, así que no podemos asegurar la convergecia del sistema por este criterio.

In [ ]:
solve(factor(det(A - eye(n) * l)), l)

**Criterio de diagonal dominante**: 4 > 2, en base al criterio, los métodos de Jacoby y Gs son convergentes.



**Criterio SOR**: Si la matriz es simétrica definida positiva y 0 < $\omega$ < 2, el método SOR converge. 

Para probar que este criterio se cumple tenemos que comprobar que el determinante de la matriz es > 0 para todo $n$. Para $n = 1, 2, 3$ los determinantes son $4, 14, 56$.

Asumamos que para una matriz de $n-1$ y $n-2$ su determinante es positivo. Veamos para el caso de $n$. Desarrollando el determinante por filas tenemos que 

$$det(A_n) = 4 \cdot det(A_{n-1}) - A_{n-2}$$

Lo de $A_{n-2}$ lo podemos hacer desarrollando un poco los términos:
$$
det\begin{pmatrix}
\color{gray}{4} & \color{gray}{1} & \color{gray}{0} & \cdots & \color{gray}{0} & \color{gray}{0} & \color{gray}{0} \\
1 & \color{gray}{4} & 1 & \cdots & 0 & 0 & 0 \\
0 & \color{gray}{1} & 4 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & \color{gray}{0} & 0 & \cdots & 4 & 1 & 0 \\
0 & \color{gray}{0} & 0 & \cdots & 1 & 4 & 1 \\
0 & \color{gray}{0} & 0 & \cdots & 0 & 1 & 4 \\
\end{pmatrix}_{n\times n} 
\rightarrow
det\begin{pmatrix}
1 & 1 & 0 & \cdots & 0 & 0 & 0 \\
0 & 4 & 1 & \cdots & 0 & 0 & 0 \\
0 & 1 & 4 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 4 & 1 & 0 \\
0 & 0 & 0 & \cdots & 1 & 4 & 1 \\
0 & 0 & 0 & \cdots & 0 & 1 & 4 \\
\end{pmatrix}_{n-1\times n-1}
=
det\begin{pmatrix}
4 & 1 & 0 & \cdots & 0 & 0 & 0 \\
1 & 4 & 1 & \cdots & 0 & 0 & 0 \\
0 & 1 & 4 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 4 & 1 & 0 \\
0 & 0 & 0 & \cdots & 1 & 4 & 1 \\
0 & 0 & 0 & \cdots & 0 & 1 & 4 \\
\end{pmatrix}_{n-2\times n-2}$$ 

Si asumimos que $\det(A_{n-2}) < \det(A_{n-1})$ entonces se cumple que $A_n > A_{n-1} > A_{n-2} > 0$ y el criterio SOR se cumple.